In [7]:
%matplotlib notebook
import os
import pickle
import numpy as np
import get_3dhst as hst
import elgtools as elg
import matplotlib.pyplot as plt
elgdir   = os.getcwd()
import jplus
import matplotlib.gridspec as gsc

tiledata = 'tiles/tiles_data_new.tsv' 
t_info = np.loadtxt(tiledata)

#mag_type = 'gpsfMags'
mag_type = 'gpsfMagsIso'
fout = '%s/out/elgs.dat' % elgdir

#j0660 = jplus.datasets.fetch_jplus_filter('J0660')
#wline = 3727.0 # OII rest-frame

#z_oii = elg.zline(wline, j0660.wave, j0660.throughput)
#print 'z-range',z_oii
z_oii = [0.73, 0.79]

hstdata = hst.select_3DHST_z(z_oii[0],z_oii[1],LineName='Ha_flux',LineMin=0.01)

LoadJPLUS = False # Load a ELG catalogue of JPLUS already built

if LoadJPLUS:
    gal_elgs = pickle.load(open(fout))
else:
    mag_excess = "AND (m.MAG_APER_3_0[jplus::rSDSS]- m.MAG_APER_3_0[jplus::J0660]) > 0"
    gal_jplus = jplus.datasets.fetch_jplus_objects(mag_type=mag_type, overwrite=False, 
                                                 object_name="jplus_pz", nchunks=20, mag_limit=[16,24.],
                                                extra_where_conds='',db='idr', allphotoz=True, upperlimparams=True)
    
      
elgtiles = np.unique(gal_jplus['tile_id'])

jtiles = jplus.datasets.fetch_jplus_tile_list(db='idr',overwrite=False)
meanAEGIS = [np.mean(hstdata['ra']), np.mean(hstdata['dec'])] # rough way to find a representative ra,dec of AEGIS

# tile closest to meanAEGIS:
# pick 4 closest tiles:
id_AEGIStile = np.argsort(elg.haversine_dist(meanAEGIS[0], meanAEGIS[1], jtiles['ra'], jtiles['dec']))
AEGIS_tile = jtiles['tileIDs'][id_AEGIStile[0:4]]


    

j-plus [INFO]: Fetching J-PLUS galaxies
j-plus [INFO]:    Loading /home/CEFCA/aaorsi/photoz/jplus_data/jplus_jplus_pz_gpsfMagsIso_idr_dual_magmax24.0.h5
j-plus [INFO]: Fetching J-PLUS Tile list
j-plus [INFO]:    Loading /home/CEFCA/aaorsi/photoz/jplus_data/objects_tile_list_idr_dual.h5


In [8]:
print jtiles.keys()
print AEGIS_tile

['ncombined', 'noise', 'min_ra', 'ADUlevel', 'tileIDs', 'texposed', 'min_dec', 'ref_tileID', 'filter', 'depth', 'effectime', 'max_dec', 'ra', 'SQL', 'date', 'max_ra', 'dec', 'fwhmg']
[28840 31975 31450 27786]


In [10]:

print AEGIS_tile, meanAEGIS


[28840 31975 31450 27786] [214.88279943688192, 52.857105586107011]


In [11]:
jplus_aegis = jplus.tools.select_object(gal_jplus, gal_jplus['tile_id'] == AEGIS_tile)
print jplus_aegis
#jplus_aegis = gal_elgs

IndexError: in the future, 0-d boolean arrays will be interpreted as a valid boolean index

In [ ]:
nhst = len(hstdata['ra'])
hst_coords = np.zeros([nhst,2])

for i in range(nhst):
    hst_coords[i,:] = [hstdata['ra'][i],hstdata['dec'][i]]

print hst_coords[0:10]
print hstdata['ra'][0:10]
hstdata['coords'] = hst_coords

In [ ]:
xmatch = elg.find_xmatches(gal_jplus, hstdata,getz = True, zcoord = 'z')


In [ ]:
print len(xmatch)

In [ ]:
print xmatch.keys()
print len(gal_jplus['coords'][:,0])

In [ ]:
plt.figure(2)

plt.plot(hstdata['coords'][:,0], hstdata['coords'][:,1],'o',color='red')
plt.plot(jplus_aegis['coords'][:,0], jplus_aegis['coords'][:,1],'o',color='green',markersize=5)
plt.xlim([hstdata['ra'].min()*.9975,hstdata['ra'].max()*1.0025])
plt.ylim([hstdata['dec'].min()*.9975,hstdata['dec'].max()*1.0025])          

In [ ]:
xmatch['tile_id']

In [ ]:
%matplotlib inline
data_tiles = np.unique(gal_jplus['tile_id'])

gs = gsc.GridSpec(4,3)
gs.update(wspace=0.0, hspace=0.0)

filternames = [ 'J0378','J0395', 'J0410','J0430','J0515', 'J0660',
               'J0861','uJAVA', 'gJAVA', 'rJAVA', 'iJAVA','zJAVA']

idf = 0
for ix in range(4):
    for iy in range(3):

        ax = plt.subplot(gs[ix,iy])
        for tile in data_tiles:
            jp = jplus.tools.select_object(gal_jplus,gal_jplus['tile_id'] == tile)
            ax.plot(jp[filternames[idf]][:,0], 1/jp[filternames[idf]][:,1], ',',color = 'gray')

        jpa = jplus.tools.select_object(gal_jplus,gal_jplus['tile_id'] == AEGIS_tile)
        ax.plot(jpa[filternames[idf]][:,0], 1/jpa[filternames[idf]][:,1], '.',color = 'red', markersize= 10,
        label='AEGIS tile' if idf == 0 else '')
    
        
        ax.text(0.8,0.85,filternames[idf],transform=ax.transAxes,fontsize=10)
        ax.legend(loc='upper left',fontsize=10)
        ax.set_xlim([16.1,23.9])
        ax.set_ylim([0,99])
        if iy >0:
            ax.set_yticklabels([])
            
        idf += 1

        